In [1]:
# Importing
# System
import os
import sys

# Computing
import mne
import numpy as np

# Plotting
import matplotlib.pyplot as plt
from surfer import Brain
from mayavi import mlab

# Local tool
sys.path.append(os.path.join('..', 'v2.0'))  # noqa
from tools.data_manager import DataManager

In [2]:
# Settings
# NameSpace

class NameSpace(object):
    # Name space
    def __init__(self, subjectID):
        # Subject ID
        self.subjectID = subjectID
        # MEG running name
        self.meg = f'MEG_{subjectID}'
        
    def load_freesurfer(self, rawpath):
        # Load freesurfer names
        # rawpath: Path of raw fif
        # Freesurfer subject
        self.freesurfer=f'RSVP_MRI_{self.subjectID}'
        # Source space
        self.src = f'{rawpath}/{self.freesurfer}-src.fif'
        self.model = f'{rawpath}/{self.freesurfer}-bem.fif'
        self.sol = f'{rawpath}/{self.freesurfer}-bem-sol.fif'
        self.trans = f'{rawpath}/{self.freesurfer}-trans.fif'

# SourceStuff
class SourceStuff(object):
    # Stuff of source analysis
    def __init__(self, ns):
        # ns: NameSpace instance
        self.ns = ns
        
    def load_fif(self):
        # Load fif files
        self.src = mne.read_source_spaces(self.ns.src)
        self.model = mne.read_bem_surfaces(self.ns.model)
        self.sol = mne.read_bem_solution(self.ns.sol)
        self.trans = mne.read_trans(self.ns.trans)
        
    def compute(self, info, epochs_baseline, loose=0.2, depth=0.8, n_jobs=48):
        # Compute inverse operator
        # info: Raw info of the MEG data
        # epochs: The epochs of interest
        
        # Compute fwd
        self.fwd = mne.make_forward_solution(info,
                                            self.trans,
                                            self.src,
                                            self.sol,
                                            eeg=False,
                                            n_jobs=n_jobs)
        
        # Compute covariance
        self.cov = mne.compute_covariance(epochs_baseline,
                                         tmax=.0,
                                         method='empirical',
                                         n_jobs=n_jobs)
        
        # Compute inverse operator
        self.inv = mne.minimum_norm.make_inverse_operator(info,
                                                          self.fwd,
                                                          self.cov,
                                                          loose=loose,
                                                          depth=depth)
        
    def estimate(self, evoked, subject_to='fsaverage', spacing=6, lambda2=1/9):
        # Estimate source activation of the [evoked]
        # Compute source estimation
        stc = mne.minimum_norm.apply_inverse(evoked,
                                                 self.inv,
                                                 lambda2=lambda2)
        
        # Compute morph
        morph = mne.compute_source_morph(src=self.src,
                                        subject_from=stc.subject,
                                        subject_to=subject_to,
                                        spacing=spacing)
        
        # Apply morph
        stc_morph = morph.apply(stc)
        
        return stc, stc_morph
    
# Freesurfer path
mne.utils.set_config('SUBJECTS_DIR', '/home/zcc/Documents/freesurfer/subjects')

# Computing parameters
n_jobs = 48

# Epochs will be filtered into bands in below
BANDS = dict(delta=(0.1, 3),
             theta=(3.5, 7.5),
             alpha=(7.5, 13),
             raw=None)

In [3]:
# Startup
for subjectID in [f'S{e:02d}' for e in range(1, 11)]:
    # ----------------------------------------------------------------------------------
    # Prepare managers
    ns = NameSpace(subjectID)
    dm = DataManager(ns.meg)
    ns.load_freesurfer(rawpath=dm.raw_dir)
    ss = SourceStuff(ns)

    # ----------------------------------------------------------------------------------
    # Load MEG data
    dm.load_epochs()
    # block_04_raw.fif may always exists
    # so, we use it to get raw.info
    raw_info = mne.io.read_raw_fif(os.path.join(dm.raw_dir, 'block_04_raw.fif')).info
    # Concancate all the epochs
    epochs, _ = dm.leave_one_session_out(includes=range(len(dm.epochs_list)), excludes=[])

    # xdawn
    # xdawn = mne.preprocessing.Xdawn(n_components=6)
    # xdawn.fit(epochs)
    epochs.apply_baseline((None, 0))
    # epochs = xdawn.apply(epochs)['1']

    # ----------------------------------------------------------------------------------
    # Compute necessaries for source analysis
    ss.load_fif()
    ss.compute(raw_info, epochs['2'])

    # ----------------------------------------------------------------------------------
    # Generate new target epochs with multiple bands
    epochs_bands = dict()
    for band in BANDS:
        _epochs = epochs['1'].copy()
        if band == 'raw':
            epochs_bands[band] = _epochs
        else:
            l_freq, h_freq = BANDS[band]
            print(l_freq, h_freq)
            epochs_bands[band] = _epochs.filter(l_freq, h_freq, n_jobs=n_jobs)

    # ----------------------------------------------------------------------------------
    # Estimate source activity
    for band in BANDS:
        stc, stc_morph = ss.estimate(epochs_bands[band].average())
        stc_morph.save(f'stc_folder/{subjectID}-{band}')

Found 8 files that ends with -epo.fif in /home/zcc/RSVP_dataset/memory/MEG_S01
Reading /home/zcc/RSVP_dataset/memory/MEG_S01/Session_1-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    1200.00 ms
        0 CTF compensation matrices available
1448 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/zcc/RSVP_dataset/memory/MEG_S01/Session_5-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    1200.00 ms
        0 CTF compensation matrices available
1461 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/zcc/RSVP_dataset/memory/MEG_S01/Session_6-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    1200.00 ms
        0 CTF compensation matrices available
1449 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/zcc/RSVP_dataset/memor

[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  11 out of  48 | elapsed:   17.2s remaining:   57.7s
[Parallel(n_jobs=48)]: Done  21 out of  48 | elapsed:   17.3s remaining:   22.2s
[Parallel(n_jobs=48)]: Done  31 out of  48 | elapsed:   17.4s remaining:    9.5s
[Parallel(n_jobs=48)]: Done  41 out of  48 | elapsed:   17.4s remaining:    3.0s


    Skipping interior check for 1447 sources that fit inside a sphere of radius   50.4 mm
    Skipping solid angle check for 0 points using Qhull


[Parallel(n_jobs=48)]: Done  48 out of  48 | elapsed:   17.8s finished
[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  11 out of  48 | elapsed:    0.1s remaining:    0.4s
[Parallel(n_jobs=48)]: Done  21 out of  48 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=48)]: Done  31 out of  48 | elapsed:    0.2s remaining:    0.1s



Setting up compensation data...
    272 out of 272 channels have the compensation set.
    Desired compensation data (3) found.
    All compensation channels found.
    Preselector created.
    Compensation data matrix created.
    Postselector created.

Composing the field computation matrix...


...

3.5 7.5
Setting up band-pass filter from 3.5 - 7.5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 3.50
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.50 Hz)
- Upper passband edge: 7.50 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 8.50 Hz)
- Filter length: 165 samples (1.650 sec)



<ipython-input-3-823c8690b665>:40: RuntimeWarning: filter_length (165) is longer than the signal (141), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_bands[band] = _epochs.filter(l_freq, h_freq, n_jobs=n_jobs)
[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 216 tasks      | elapsed:    0.3s
[Parallel(n_jobs=48)]: Done 1504 tasks      | elapsed:    0.8s
[Parallel(n_jobs=48)]: Done 10320 tasks      | elapsed:    2.4s
[Parallel(n_jobs=48)]: Done 22992 tasks      | elapsed:    4.0s
[Parallel(n_jobs=48)]: Done 37968 tasks      | elapsed:    5.7s
[Parallel(n_jobs=48)]: Done 55248 tasks      | elapsed:    7.9s
[Parallel(n_jobs=48)]: Done 74832 tasks      | elapsed:   10.7s
[Parallel(n_jobs=48)]: Done 96720 tasks      | elapsed:   13.4s
[Parallel(n_jobs=48)]: Done 120912 tasks      | elapsed:   16.1s
[Parallel(n_jobs=48)]: Done 147408 tasks    

7.5 13
Setting up band-pass filter from 7.5 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.50
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.50 Hz)
- Upper passband edge: 13.00 Hz
- Upper transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 14.62 Hz)
- Filter length: 165 samples (1.650 sec)



<ipython-input-3-823c8690b665>:40: RuntimeWarning: filter_length (165) is longer than the signal (141), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_bands[band] = _epochs.filter(l_freq, h_freq, n_jobs=n_jobs)
[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 216 tasks      | elapsed:    0.3s
[Parallel(n_jobs=48)]: Done 1488 tasks      | elapsed:    0.8s
[Parallel(n_jobs=48)]: Done 6672 tasks      | elapsed:    1.6s
[Parallel(n_jobs=48)]: Done 15312 tasks      | elapsed:    3.0s
[Parallel(n_jobs=48)]: Done 30288 tasks      | elapsed:    5.2s
[Parallel(n_jobs=48)]: Done 47568 tasks      | elapsed:    7.5s
[Parallel(n_jobs=48)]: Done 67152 tasks      | elapsed:    9.8s
[Parallel(n_jobs=48)]: Done 89040 tasks      | elapsed:   12.6s
[Parallel(n_jobs=48)]: Done 113232 tasks      | elapsed:   15.4s
[Parallel(n_jobs=48)]: Done 139728 tasks     

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 560
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 272 (0 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Applying inverse operator to "1"...
    Picked 272 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  50.9% variance
    Combining the current components...
    dSPM...
[done]
Writing STC to disk...
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 560
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 272 (0 small eigenvalues omitted)
    Computing noise-normalization factors (dSP

In [4]:
# Startup
for subjectID in [f'S{e:02d}' for e in range(1, 11)]:
    # ----------------------------------------------------------------------------------
    # Prepare managers
    ns = NameSpace(subjectID)
    dm = DataManager(ns.meg)
    ns.load_freesurfer(rawpath=dm.raw_dir)
    ss = SourceStuff(ns)

    # ----------------------------------------------------------------------------------
    # Load MEG data
    dm.load_epochs()
    # block_04_raw.fif may always exists
    # so, we use it to get raw.info
    raw_info = mne.io.read_raw_fif(os.path.join(dm.raw_dir, 'block_04_raw.fif')).info
    # Concancate all the epochs
    epochs, _ = dm.leave_one_session_out(includes=range(len(dm.epochs_list)), excludes=[])

    # xdawn
    xdawn = mne.preprocessing.Xdawn(n_components=6)
    xdawn.fit(epochs)
    epochs.apply_baseline((None, 0))
    epochs = xdawn.apply(epochs)['1']

    # ----------------------------------------------------------------------------------
    # Compute necessaries for source analysis
    ss.load_fif()
    ss.compute(raw_info, epochs['2'])

    # ----------------------------------------------------------------------------------
    # Generate new target epochs with multiple bands
    epochs_bands = dict()
    for band in BANDS:
        _epochs = epochs['1'].copy()
        if band == 'raw':
            epochs_bands[band] = _epochs
        else:
            l_freq, h_freq = BANDS[band]
            print(l_freq, h_freq)
            epochs_bands[band] = _epochs.filter(l_freq, h_freq, n_jobs=n_jobs)

    # ----------------------------------------------------------------------------------
    # Estimate source activity
    for band in BANDS:
        stc, stc_morph = ss.estimate(epochs_bands[band].average())
        stc_morph.save(f'xdawn_stc_folder/{subjectID}-{band}')

Found 8 files that ends with -epo.fif in /home/zcc/RSVP_dataset/memory/MEG_S01
Reading /home/zcc/RSVP_dataset/memory/MEG_S01/Session_1-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    1200.00 ms
        0 CTF compensation matrices available
1448 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/zcc/RSVP_dataset/memory/MEG_S01/Session_5-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    1200.00 ms
        0 CTF compensation matrices available
1461 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/zcc/RSVP_dataset/memory/MEG_S01/Session_6-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...    1200.00 ms
        0 CTF compensation matrices available
1449 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Reading /home/zcc/RSVP_dataset/memor

[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  11 out of  48 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=48)]: Done  21 out of  48 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=48)]: Done  31 out of  48 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=48)]: Done  41 out of  48 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=48)]: Done  48 out of  48 | elapsed:    0.2s finished


    Skipping solid angle check for 0 points using Qhull


[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  11 out of  48 | elapsed:    0.1s remaining:    0.4s
[Parallel(n_jobs=48)]: Done  21 out of  48 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=48)]: Done  31 out of  48 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=48)]: Done  41 out of  48 | elapsed:    0.2s remaining:    0.0s



Setting up compensation data...
    272 out of 272 channels have the compensation set.
    Desired compensation data (3) found.
    All compensation channels found.
    Preselector created.
    Compensation data matrix created.
    Postselector created.

Composing the field computation matrix...


...

3.5 7.5
Setting up band-pass filter from 3.5 - 7.5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 3.50
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 2.50 Hz)
- Upper passband edge: 7.50 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 8.50 Hz)
- Filter length: 165 samples (1.650 sec)



<ipython-input-4-185f082b875e>:40: RuntimeWarning: filter_length (165) is longer than the signal (141), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_bands[band] = _epochs.filter(l_freq, h_freq, n_jobs=n_jobs)
[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 216 tasks      | elapsed:    0.3s
[Parallel(n_jobs=48)]: Done 1488 tasks      | elapsed:    0.7s
[Parallel(n_jobs=48)]: Done 6672 tasks      | elapsed:    1.5s
[Parallel(n_jobs=48)]: Done 21456 tasks      | elapsed:    3.9s
[Parallel(n_jobs=48)]: Done 51408 tasks      | elapsed:    7.3s
[Parallel(n_jobs=48)]: Done 85968 tasks      | elapsed:   11.5s
[Parallel(n_jobs=48)]: Done 125136 tasks      | elapsed:   16.2s
[Parallel(n_jobs=48)]: Done 149088 tasks      | elapsed:   18.7s
[Parallel(n_jobs=48)]: Done 152320 out of 152320 | elapsed:   19.2s finished


7.5 13
Setting up band-pass filter from 7.5 - 13 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.50
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.50 Hz)
- Upper passband edge: 13.00 Hz
- Upper transition bandwidth: 3.25 Hz (-6 dB cutoff frequency: 14.62 Hz)
- Filter length: 165 samples (1.650 sec)



<ipython-input-4-185f082b875e>:40: RuntimeWarning: filter_length (165) is longer than the signal (141), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_bands[band] = _epochs.filter(l_freq, h_freq, n_jobs=n_jobs)
[Parallel(n_jobs=48)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done  24 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 216 tasks      | elapsed:    0.2s
[Parallel(n_jobs=48)]: Done 1488 tasks      | elapsed:    0.7s
[Parallel(n_jobs=48)]: Done 6672 tasks      | elapsed:    1.6s
[Parallel(n_jobs=48)]: Done 15312 tasks      | elapsed:    3.0s
[Parallel(n_jobs=48)]: Done 30288 tasks      | elapsed:    5.1s
[Parallel(n_jobs=48)]: Done 47568 tasks      | elapsed:    7.2s
[Parallel(n_jobs=48)]: Done 67152 tasks      | elapsed:    9.6s
[Parallel(n_jobs=48)]: Done 89040 tasks      | elapsed:   12.7s
[Parallel(n_jobs=48)]: Done 113232 tasks      | elapsed:   15.8s
[Parallel(n_jobs=48)]: Done 139728 tasks     

Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 560
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 6 (266 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Applying inverse operator to "1"...
    Picked 272 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  97.6% variance
    Combining the current components...
    dSPM...
[done]
Writing STC to disk...
[done]
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 560
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 6 (266 small eigenvalues omitted)
    Computing noise-normalization factors (dSP

In [ ]:
# Load stc
subjectID = 'S05'
band = 'alpha'
stc = mne.read_source_estimate(f'stc_folder/{subjectID}-{band}')

# Interactive View
# Settings
mne.viz.set_3d_backend('pyvista')
surfer_kwargs = dict(hemi='both',
                     time_unit='s',
                     size=(800, 800),
                     smoothing_steps=10,
                     time_viewer=True,
                     subject='fsaverage')
# Plot
# This can not be operated using VS code
brain = stc.plot(**surfer_kwargs)

In [7]:
labels_aparc = mne.read_labels_from_annot('fsaverage', 'aparc', 'both')
labels_visuotopic = mne.read_labels_from_annot('fsaverage', 'PALS_B12_Visuotopic', 'both')

display(labels_aparc)
display(labels_visuotopic)

Reading labels from parcellation...
   read 35 labels from /home/zcc/Documents/freesurfer/subjects/fsaverage/label/lh.aparc.annot
   read 34 labels from /home/zcc/Documents/freesurfer/subjects/fsaverage/label/rh.aparc.annot
Reading labels from parcellation...
   read 16 labels from /home/zcc/Documents/freesurfer/subjects/fsaverage/label/lh.PALS_B12_Visuotopic.annot
   read 27 labels from /home/zcc/Documents/freesurfer/subjects/fsaverage/label/rh.PALS_B12_Visuotopic.annot


[<Label  |  fsaverage, 'bankssts-lh', lh : 2137 vertices>,
 <Label  |  fsaverage, 'bankssts-rh', rh : 2196 vertices>,
 <Label  |  fsaverage, 'caudalanteriorcingulate-lh', lh : 1439 vertices>,
 <Label  |  fsaverage, 'caudalanteriorcingulate-rh', rh : 1608 vertices>,
 <Label  |  fsaverage, 'caudalmiddlefrontal-lh', lh : 3736 vertices>,
 <Label  |  fsaverage, 'caudalmiddlefrontal-rh', rh : 3494 vertices>,
 <Label  |  fsaverage, 'cuneus-lh', lh : 1630 vertices>,
 <Label  |  fsaverage, 'cuneus-rh', rh : 1638 vertices>,
 <Label  |  fsaverage, 'entorhinal-lh', lh : 1102 vertices>,
 <Label  |  fsaverage, 'entorhinal-rh', rh : 902 vertices>,
 <Label  |  fsaverage, 'frontalpole-lh', lh : 272 vertices>,
 <Label  |  fsaverage, 'frontalpole-rh', rh : 369 vertices>,
 <Label  |  fsaverage, 'fusiform-lh', lh : 4714 vertices>,
 <Label  |  fsaverage, 'fusiform-rh', rh : 4661 vertices>,
 <Label  |  fsaverage, 'inferiorparietal-lh', lh : 7871 vertices>,
 <Label  |  fsaverage, 'inferiorparietal-rh', rh : 9

[<Label  |  fsaverage, '???-lh', lh : 116427 vertices>,
 <Label  |  fsaverage, '???-rh', rh : 117864 vertices>,
 <Label  |  fsaverage, '????-rh', rh : 117864 vertices>,
 <Label  |  fsaverage, '???????-rh', rh : 117864 vertices>,
 <Label  |  fsaverage, 'AUDITORY-lh', lh : 930 vertices>,
 <Label  |  fsaverage, 'AUDITORY-rh', rh : 955 vertices>,
 <Label  |  fsaverage, 'GYRUS-rh', rh : 117864 vertices>,
 <Label  |  fsaverage, 'LANDMARK.MT-rh', rh : 955 vertices>,
 <Label  |  fsaverage, 'LANDMARK.V2d-rh', rh : 1867 vertices>,
 <Label  |  fsaverage, 'LANDMARK.V2v-rh', rh : 1867 vertices>,
 <Label  |  fsaverage, 'MEDIAL.WALL-lh', lh : 11608 vertices>,
 <Label  |  fsaverage, 'MEDIAL.WALL-rh', rh : 11312 vertices>,
 <Label  |  fsaverage, 'NonVisuotopic-lh', lh : 22715 vertices>,
 <Label  |  fsaverage, 'NonVisuotopic-rh', rh : 21697 vertices>,
 <Label  |  fsaverage, 'SUL.HF-rh', rh : 955 vertices>,
 <Label  |  fsaverage, 'SUL.ISS-rh', rh : 955 vertices>,
 <Label  |  fsaverage, 'Thresh_Normal_dee

In [9]:
stc_averaged_bands = dict()

for band in BANDS:
    # Generate mean stc
    stcs = []
    for subjectID in [f'S{e:02d}' for e in range(1, 11)]:
        stcs.append(mne.read_source_estimate(f'xdawn_stc_folder/{subjectID}-{band}'))
    print(f'Got {len(stcs)} stcs in band of {band}')

    stc_averaged = stcs[0]
    for j, stc in enumerate(stcs):
        if j== 0:
            continue
        stc_averaged.data += stc.data

    stc_averaged.data /= len(stcs)
    
    stc_averaged_bands[band] = stc_averaged
    
stc_averaged_bands

Got 10 stcs in band of delta
Got 10 stcs in band of theta
Got 10 stcs in band of alpha
Got 10 stcs in band of raw


{'delta': <SourceEstimate  |  81924 vertices, tmin : -200.0 (ms), tmax : 1200.0000000000002 (ms), tstep : 10.0 (ms), data shape : (81924, 141)>,
 'theta': <SourceEstimate  |  81924 vertices, tmin : -200.0 (ms), tmax : 1200.0000000000002 (ms), tstep : 10.0 (ms), data shape : (81924, 141)>,
 'alpha': <SourceEstimate  |  81924 vertices, tmin : -200.0 (ms), tmax : 1200.0000000000002 (ms), tstep : 10.0 (ms), data shape : (81924, 141)>,
 'raw': <SourceEstimate  |  81924 vertices, tmin : -200.0 (ms), tmax : 1200.0000000000002 (ms), tstep : 10.0 (ms), data shape : (81924, 141)>}

In [13]:
mne.viz.set_3d_backend('pyvista')
band = 'theta'
for band in BANDS:
    # Get stc
    stc = stc_averaged_bands[band]

    # Compute lims of colormap
    alldata = sorted(stc.data.ravel(), reverse=True)
    n = len(alldata)
    lims = [alldata[int(n * r)] for r in [0.005, 0.001, 0]]

    labels = []
    for lab in labels_aparc:
        try:
            _stc = stc.in_label(lab)
        except ValueError:
            # Incase can not find valid stc vertex
            continue

        # Test if the signals in the [lab] is strong enough
        data = _stc.data
        data = data[np.max(data, axis=1) > lims[1]]
        if len(data) < 500:
            # Continue, if the signal is too weak
            continue

        labels.append(lab)

    print(f'Found {len(labels)} valid labels.')

    # Display settings
    clim = dict(kind='value', lims=lims)
    surfer_kwargs = dict(hemi='both',
                         clim=clim,
                         title=band.title(),
                         initial_time=0.3,
                         time_unit='s',
                         size=(800, 800),
                         smoothing_steps=10,
                         time_viewer=True,
                         subject='fsaverage')

    # Display
    brain = stc.plot(**surfer_kwargs)
#     for lab in labels:
#         print(lab)
#         brain.add_label(lab, borders=True)

Found 0 valid labels.
Found 1 valid labels.
Found 2 valid labels.
Found 0 valid labels.


Traceback (most recent call last):
  File "/home/zcc/anaconda3/lib/python3.7/site-packages/mne/viz/_brain/_timeviewer.py", line 24, in safe_event
    return fun(*args, **kwargs)
  File "/home/zcc/anaconda3/lib/python3.7/site-packages/mne/viz/_brain/_timeviewer.py", line 389, in keyPressEvent
    callback = self.key_bindings.get(event.text())
AttributeError: 'NoneType' object has no attribute 'get'
